In [186]:
import numpy as np
import numpy.linalg as la
import pandas as pd
from scipy.io.arff import loadarff 
from sklearn.decomposition import PCA
import scipy

In [222]:
raw_data = loadarff('electricity-normalized.arff')
df = pd.DataFrame(raw_data[0])
df['target'] = np.where(df['class'] == b'UP', 1, 0)
df = pd.get_dummies(df.drop(columns='class'))

In [223]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45312 entries, 0 to 45311
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       45312 non-null  float64
 1   period     45312 non-null  float64
 2   nswprice   45312 non-null  float64
 3   nswdemand  45312 non-null  float64
 4   vicprice   45312 non-null  float64
 5   vicdemand  45312 non-null  float64
 6   transfer   45312 non-null  float64
 7   target     45312 non-null  int64  
 8   day_b'1'   45312 non-null  uint8  
 9   day_b'2'   45312 non-null  uint8  
 10  day_b'3'   45312 non-null  uint8  
 11  day_b'4'   45312 non-null  uint8  
 12  day_b'5'   45312 non-null  uint8  
 13  day_b'6'   45312 non-null  uint8  
 14  day_b'7'   45312 non-null  uint8  
dtypes: float64(7), int64(1), uint8(7)
memory usage: 3.1 MB


**Date**: date between 7 May 1996 to 5 December 1998. Here normalized between 0 and 1

**Day**: day of the week (1-7)

**Period**: time of the measurement (1-48) in half hour intervals over 24 hours. Here normalized between 0 and 1

**NSWprice**: New South Wales electricity price, normalized between 0 and 1

**NSWdemand**: New South Wales electricity demand, normalized between 0 and 1

**VICprice**: Victoria electricity price, normalized between 0 and 1

**VICdemand**: Victoria electricity demand, normalized between 0 and 1

**transfer**: scheduled electricity transfer between both states, normalized between 0 and 1

In [218]:
first = df.query('date < 0.45')
second = df.query('date > 0.98')
v1 = np.sqrt(np.sum((scipy.linalg.svdvals(first) - scipy.linalg.svdvals(second))**2))
U, S, VT = la.svd(first)
U_s, S_s, VT_s = la.svd(second)
print(v1)

first = df.query('date < 0.45')
second = df.query('date > 0.90')
v2 = np.sqrt(np.sum((scipy.linalg.svdvals(first) - scipy.linalg.svdvals(second))**2))
print(v2)

first = df.query('date < 0.60')
second = df.query('date > 0.60 and date < 0.90')
U, S, VT = la.svd(first)
U_s, S_s, VT_s = la.svd(second)
v3 = np.sqrt(np.sum((scipy.linalg.svdvals(first) - scipy.linalg.svdvals(second))**2))
print(v3)

first = df.query('date < 0.45')
second = df.query('date > 0.90')

190.37229263333418
97.65797924750096
77.65509729067612


In [209]:
np.round(VT[0], 2)

array([-0.17, -0.47, -0.06, -0.37, -0.  , -0.36, -0.36, -0.49, -0.13,
       -0.12, -0.12, -0.12, -0.12, -0.12, -0.12])

In [208]:
np.round(VT_s[0], 2)

array([-0.64, -0.34, -0.04, -0.3 , -0.  , -0.29, -0.38, -0.3 , -0.11,
       -0.11,  0.  , -0.11, -0.1 , -0.1 , -0.11])

In [217]:
print(np.dot(VT_s[0], VT[0]))
print(np.dot(VT_s[1], VT[1]))
print(np.dot(VT_s[2], VT[2]))
print(np.dot(VT_s[3], VT[3]))

0.8526869904905549
0.8082442362531134
-0.7138330007722307
0.07640734246983011


In [219]:
print(np.dot(VT_s[0], VT[0]))
print(np.dot(VT_s[1], VT[1]))
print(np.dot(VT_s[2], VT[2]))
print(np.dot(VT_s[3], VT[3]))

0.9196151271197981
0.924274411102636
-0.832632420087531
0.014504586130451452


In [189]:
X = first.drop(columns=['target', 'date'])
y = first['target']
weights = la.inv(X.T @ X) @ X.T @ y

In [190]:
test_X = second.drop(columns=['target', 'date'])
test_y = second['target']

In [191]:
weights.index = X.columns

In [192]:
corr_compare = pd.DataFrame({'first':first.corr().round(2)['target'], 'second':second.corr().round(2)['target']})
corr_compare

,first,second
date,0.01,-0.05
period,0.17,0.11
nswprice,0.58,0.54
nswdemand,0.34,0.41
vicprice,0.21,0.15
vicdemand,0.15,0.38
transfer,-0.09,-0.17
target,1.00,1.00
day_b'1',0.07,0.01
day_b'2',-0.01,0.14


In [193]:
weights.loc['vicdemand'] *= 1.23
# weights.loc["day_b'2'"] *= 1.23
pred_y = test_X.values @ weights

In [194]:
print(f"train error: {np.mean(np.where(X.values @ weights < 0.5, 0, 1) == y)}")
print(f"test error: {np.mean(np.where(pred_y < 0.5, 0, 1) == test_y)}")

train error: 0.8180423594615994
test error: 0.8333333333333334


In [82]:
pca = PCA(n_components=1)
first_x = pca.fit_transform(first.iloc[:,:4])
second_x = pca.fit_transform(second.iloc[:,:4])
# second_x_test = pca.fit_transform(second_test.iloc[:,:4])

In [85]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4, random_state=0).fit(first)

In [87]:
first['k_group'] = kmeans.labels_
second['group_pred'] = kmeans.predict(second)

/var/folders/yt/s603nhzn7w5c433sz6fhky740000gn/T/ipykernel_1217/3267758018.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first['k_group'] = kmeans.labels_
/var/folders/yt/s603nhzn7w5c433sz6fhky740000gn/T/ipykernel_1217/3267758018.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second['group_pred'] = kmeans.predict(second)


In [88]:
print(first.k_group.value_counts(normalize=True))
print(second.group_pred.value_counts(normalize=True))

2    0.387767
3    0.324822
1    0.144893
0    0.142518
Name: k_group, dtype: float64
2    0.400165
3    0.312706
1    0.148515
0    0.138614
Name: group_pred, dtype: float64


In [89]:
(first.corr().round(2) - second.corr().round(2))['target']

date          0.06
day_b'1'      0.06
day_b'2'     -0.15
day_b'3'     -0.02
day_b'4'      0.03
day_b'5'     -0.01
day_b'6'      0.01
day_b'7'      0.07
group_pred     NaN
k_group        NaN
nswdemand    -0.07
nswprice      0.04
period        0.06
target        0.00
transfer      0.08
vicdemand    -0.23
vicprice      0.06
Name: target, dtype: float64

In [37]:
def H(p, q):
  # distance between p an d
  # p and q are np array probability distributions
  n = len(p)
  sum = 0.0
  for i in range(n):
    sum += (np.sqrt(p[i]) - np.sqrt(q[i]))**2
  result = (1.0 / np.sqrt(2.0)) * np.sqrt(sum)
  return result